In [ ]:
import pandas as pd
import numpy as np
import os
from preprocessing.utils import create_ehr_case_identification_column, create_registry_case_identification_column
from preprocessing.variable_assembly.variable_database_assembly import load_data_from_main_dir

In [ ]:
raw_ehr_data_dir = '/Users/jk1/stroke_datasets/stroke_unit_dataset/per_value/Extraction_20221117'
patient_selection_path = '/Users/jk1/temp/opsum_extraction_output/high_frequency_data_patient_selection_with_details.csv'
out_dir = '/Users/jk1/temp/opsum_extraction_output/ehr_extraction_errors/Extraction_20221117'

In [ ]:
lab_file_start = 'labo'
lab_df = load_data_from_main_dir(raw_ehr_data_dir, lab_file_start)
lab_df['eds_end_4digit'] = lab_df['eds_end_4digit'].str.zfill(4)
lab_df['case_admission_id'] = create_ehr_case_identification_column(lab_df)


In [ ]:
scales_file_start = 'scale'
scales_df = load_data_from_main_dir(raw_ehr_data_dir, scales_file_start)
scales_df['eds_end_4digit'] = scales_df['eds_end_4digit'].str.zfill(4)
scales_df['case_admission_id'] = create_ehr_case_identification_column(scales_df)


In [ ]:
ventilation_file_start = 'ventilation'
ventilation_df = load_data_from_main_dir(raw_ehr_data_dir, ventilation_file_start)
ventilation_df['eds_end_4digit'] = ventilation_df['eds_end_4digit'].str.zfill(4)
ventilation_df['case_admission_id'] = create_ehr_case_identification_column(ventilation_df)


In [ ]:
vitals_file_start = 'patientvalue'
vitals_df = load_data_from_main_dir(raw_ehr_data_dir, vitals_file_start)
vitals_df['eds_end_4digit'] = vitals_df['eds_end_4digit'].str.zfill(4)
vitals_df['case_admission_id'] = create_ehr_case_identification_column(vitals_df)

In [ ]:
vitals_df.head(1000)

In [ ]:
pid = '97063889'
vitals_df[vitals_df['patient_id'] == pid].head()

In [ ]:
cid_from_ehr = pd.concat([vitals_df['case_admission_id'], lab_df['case_admission_id'], scales_df['case_admission_id'], ventilation_df['case_admission_id']]).unique()
cid_from_ehr

In [ ]:
patient_selection_df = pd.read_csv(patient_selection_path, dtype=str)
patient_selection_df['case_admission_id'] = create_registry_case_identification_column(patient_selection_df)
cids_in_selection = set(patient_selection_df['case_admission_id'].unique())

In [ ]:
# find cids in selection but not in ehr
missing_cids = set(cids_in_selection).difference(set(cid_from_ehr))

In [ ]:
missing_patients_info = patient_selection_df[patient_selection_df['case_admission_id'].isin(missing_cids)]
missing_patients_info

In [ ]:
dt_format = '%Y%m%d'
missing_patients_info['delta_stroke_to_admission_days'] = pd.to_datetime(missing_patients_info['Stroke onset date'], format=dt_format) - pd.to_datetime(missing_patients_info['Arrival at hospital'], format=dt_format)

In [ ]:
missing_patients_info.to_csv(os.path.join(out_dir, 'patients_missing_in_EHR_extraction.tsv'), index=False, sep='\t')